In [2]:
! pip install sentence-transformers tqdm

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import json
import os
import re
import hashlib
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
from collections import defaultdict

# -------------------------------
# 1️⃣ Utility functions
# -------------------------------

def normalize_text(text: str) -> str:
    """Normalize text for hashing and comparison."""
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)            # collapse whitespace
    text = re.sub(r'[^\w\s]', '', text)         # remove punctuation
    return text.strip()

def compute_hash(text: str) -> str:
    """Generate a unique hash ID for dedup tracking."""
    return hashlib.md5(normalize_text(text).encode()).hexdigest()

# -------------------------------
# 2️⃣ Load and group by category
# -------------------------------




C:\Users\madha\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\madha\AppData\Roaming\Python\Python311\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Users\madha\AppData\Roaming\Python\Python311\site-p

In [6]:
input_file = "C:/Users/madha/vsCode/Github/legal-assist-rag/data/test.json"  # your JSON file
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Group all items by their category
category_data = defaultdict(list)
for item in data:
    category = item.get("query-category", "Unknown").strip()
    category_data[category].append(item)

print(f"Found {len(category_data)} categories:")
for cat, items in category_data.items():
    print(f" - {cat}: {len(items)} records")

# -------------------------------
# 3️⃣ Deduplication setup
# -------------------------------

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

output_dir = "deduplicated_data"
os.makedirs(output_dir, exist_ok=True)

# -------------------------------
# 4️⃣ Process category-wise
# -------------------------------

for category, items in category_data.items():
    print(f"\n🔹 Processing category: {category}")

    # Step 1: remove exact duplicates
    seen_hashes = set()
    unique_items = []
    for item in items:
        qtext = item.get("query-text", "")
        qhash = compute_hash(qtext)
        if qhash not in seen_hashes:
            unique_items.append(item)
            seen_hashes.add(qhash)

    print(f"   ➜ After exact dedup: {len(unique_items)} items")

    # Step 2: semantic deduplication
    query_texts = [i["query-text"] for i in unique_items]
    embeddings = model.encode(query_texts, convert_to_tensor=True, show_progress_bar=True)

    to_remove = set()
    threshold = 0.85  # tune this based on test results

    # Compare each query with the rest
    for i in tqdm(range(len(unique_items)), desc=f"Semantic dedup ({category})"):
        if i in to_remove:
            continue
        for j in range(i + 1, len(unique_items)):
            if j in to_remove:
                continue
            sim = util.cos_sim(embeddings[i], embeddings[j]).item()
            if sim > threshold:
                # Mark j as duplicate (keep the first one)
                to_remove.add(j)

    final_items = [item for idx, item in enumerate(unique_items) if idx not in to_remove]

    print(f"   ➜ After semantic dedup: {len(final_items)} items")

    # Step 3: save category-wise cleaned data
    output_path = os.path.join(output_dir, f"{category.replace(' ', '_')}_clean.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(final_items, f, ensure_ascii=False, indent=4)

    print(f"   ✅ Saved cleaned file: {output_path}")

Found 9 categories:
 - Civil Law: 87 records
 - Family Law: 307 records
 - Criminal Law: 134 records
 - Labour: 32 records
 - Property Law: 224 records
 - Business Law: 17 records
 - Consumer Law: 2 records
 - Constitutional Law: 5 records
 - Taxation: 4 records

🔹 Processing category: Civil Law
   ➜ After exact dedup: 87 items


Semantic dedup (Civil Law): 100%|██████████| 87/87 [00:00<00:00, 196.77it/s]


   ➜ After semantic dedup: 87 items
   ✅ Saved cleaned file: deduplicated_data\Civil_Law_clean.json

🔹 Processing category: Family Law
   ➜ After exact dedup: 307 items


Semantic dedup (Family Law): 100%|██████████| 307/307 [00:05<00:00, 52.92it/s] 


   ➜ After semantic dedup: 307 items
   ✅ Saved cleaned file: deduplicated_data\Family_Law_clean.json

🔹 Processing category: Criminal Law
   ➜ After exact dedup: 134 items


...

   ➜ After semantic dedup: 2 items
   ✅ Saved cleaned file: deduplicated_data\Consumer_Law_clean.json

🔹 Processing category: Constitutional Law
   ➜ After exact dedup: 5 items


Semantic dedup (Constitutional Law): 100%|██████████| 5/5 [00:00<00:00, 3042.44it/s]


   ➜ After semantic dedup: 5 items
   ✅ Saved cleaned file: deduplicated_data\Constitutional_Law_clean.json

🔹 Processing category: Taxation
   ➜ After exact dedup: 4 items


Semantic dedup (Taxation): 100%|██████████| 4/4 [00:00<00:00, 3123.09it/s]

   ➜ After semantic dedup: 4 items
   ✅ Saved cleaned file: deduplicated_data\Taxation_clean.json
